In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [7]:
df = pd.read_excel("yearly_batting_2000_2021.xlsx")

In [8]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [10]:
df['BB+HBP'] = df.apply(lambda row: row['BB']+row['HBP'], axis=1)

In [14]:
df['1B'] = df.apply(lambda row: row['H']-row['2B']-row['3B']-row['HR'], axis=1)

In [29]:
df.head()

,YEAR,TM,#BAT,BATAGE,RPG,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB,BB+HBP,1B
0,2000,Anaheim Angels,45,27.6,5.33,162,6373,5628,864,1574,309,34,236,837,93,52,608,1024,0.280,0.352,0.472,0.825,105,2659,126,47,47,43,43,1173,655,995
1,2000,Arizona Diamondbacks,41,30.8,4.89,162,6241,5527,792,1466,282,44,179,756,97,44,535,975,0.265,0.333,0.429,0.763,88,2373,114,59,61,58,37,1128,594,961
2,2000,Atlanta Braves,47,30.8,5.00,162,6275,5489,810,1490,274,26,179,758,148,56,595,1010,0.271,0.346,0.429,0.775,95,2353,127,59,87,45,38,1192,654,1011
3,2000,Baltimore Orioles,50,32.1,4.90,162,6238,5549,794,1508,310,22,184,750,126,65,558,900,0.272,0.341,0.435,0.776,100,2414,148,49,27,54,34,1129,607,992
4,2000,Boston Red Sox,52,29.3,4.89,162,6371,5630,792,1503,316,32,167,755,43,30,611,1019,0.267,0.341,0.423,0.764,90,2384,115,42,40,48,40,1226,653,988


In [33]:
df.dropna(inplace=True)

In [34]:
df.TM.unique()

array(['Anaheim Angels', 'Arizona Diamondbacks', 'Atlanta Braves',
       'Baltimore Orioles', 'Boston Red Sox', 'Chicago Cubs',
       'Chicago White Sox', 'Cincinnati Reds', 'Cleveland Indians',
       'Colorado Rockies', 'Detroit Tigers', 'Florida Marlins',
       'Houston Astros', 'Kansas City Royals', 'Los Angeles Dodgers',
       'Milwaukee Brewers', 'Minnesota Twins', 'Montreal Expos',
       'New York Mets', 'New York Yankees', 'Oakland Athletics',
       'Philadelphia Phillies', 'Pittsburgh Pirates', 'San Diego Padres',
       'Seattle Mariners', 'San Francisco Giants', 'St. Louis Cardinals',
       'Tampa Bay Devil Rays', 'Texas Rangers', 'Toronto Blue Jays',
       'Los Angeles Angels of Anaheim', 'Washington Nationals',
       'Tampa Bay Rays', 'Miami Marlins', 'Los Angeles Angels'],
      dtype=object)

In [35]:
df_regression = df[['BB+HBP', '1B', '2B', '3B', 'HR', 'SB', 'CS', 'R']]

In [36]:
X = df_regression.loc[:, df_regression.columns != 'R']

In [37]:
X = sm.add_constant(X)

In [71]:
X.head()

,const,BB+HBP,1B,2B,3B,HR,SB,CS
0,1.0,655,995,309,34,236,93,52
1,1.0,594,961,282,44,179,97,44
2,1.0,654,1011,274,26,179,148,56
3,1.0,607,992,310,22,184,126,65
4,1.0,653,988,316,32,167,43,30


In [39]:
y = pd.DataFrame(df_regression['R'])

In [40]:
y.head()

,R
0,864
1,792
2,810
3,794
4,792


In [41]:
model = sm.OLS(y, X).fit()

In [42]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      R   R-squared:                       0.931
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                     1266.
Date:                Fri, 13 May 2022   Prob (F-statistic):               0.00
Time:                        08:22:31   Log-Likelihood:                -3242.7
No. Observations:                 660   AIC:                             6501.
Df Residuals:                     652   BIC:                             6537.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -55.8191      8.653     -6.451      0.000     -72.810     -38.828
BB+HBP         0.2484      0.020     12.341      0.000       0.209       0.288
1B             0.2527      0.016     15.430      0.000       0.221       0.285
2B             0.5950      0.053     11.173      0.000       0.490       0.700
3B             0.9069      0.160      5.685      0.000       0.594       1.220
HR             1.2692      0.042     30.187      0.000       1.187       1.352
SB             0.1625      0.055      2.933      0.003       0.054       0.271
CS            -0.4820      0.153     -3.156      0.002      -0.782      -0.182
==============================================================================
Omnibus:                       19.675   Durbin-Watson:                   1.664
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               22.823
Skew:                           0.349   Prob(JB):                     1.11e-05
Kurtosis:                       3.585   Cond. No.                     7.64e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.64e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [57]:
def predicted_run(row):
    return -55.819 + (0.253 * row['1B']) + (0.595 * row['2B']) + (0.907 * row['3B']) + (1.269 * row['HR']) + (0.248 * row['BB+HBP']) + (0.163 * row['SB']) - (0.482 * row['CS'])

In [65]:
df.head()

,YEAR,TM,#BAT,BATAGE,RPG,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB,BB+HBP,1B,predicted_runs,absolute_error
0,2000,Anaheim Angels,45,27.6,5.33,162,6373,5628,864,1574,309,34,236,837,93,52,608,1024,0.280,0.352,0.472,0.825,105,2659,126,47,47,43,43,1173,655,995,862.628,1.372
1,2000,Arizona Diamondbacks,41,30.8,4.89,162,6241,5527,792,1466,282,44,179,756,97,44,535,975,0.265,0.333,0.429,0.763,88,2373,114,59,61,58,37,1128,594,961,764.078,27.922
2,2000,Atlanta Braves,47,30.8,5.00,162,6275,5489,810,1490,274,26,179,758,148,56,595,1010,0.271,0.346,0.429,0.775,95,2353,127,59,87,45,38,1192,654,1011,773.051,36.949
3,2000,Baltimore Orioles,50,32.1,4.90,162,6238,5549,794,1508,310,22,184,750,126,65,558,900,0.272,0.341,0.435,0.776,100,2414,148,49,27,54,34,1129,607,992,772.801,21.199
4,2000,Boston Red Sox,52,29.3,4.89,162,6371,5630,792,1503,316,32,167,755,43,30,611,1019,0.267,0.341,0.423,0.764,90,2384,115,42,40,48,40,1226,653,988,777.605,14.395


In [66]:
def predicted_run_v2(row):
    return -55.819 + (0.253 * row['1B']) + (0.595 * row['2B']) + (0.907 * row['3B']) + (1.269 * row['HR']) + (0.248 * row['BB+HBP']) + (0.163 * row['SB'])

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 0 to 680
Data columns (total 33 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   YEAR            660 non-null    int64  
 1   TM              660 non-null    object 
 2   #BAT            660 non-null    int64  
 3   BATAGE          660 non-null    float64
 4   RPG             660 non-null    float64
 5   G               660 non-null    int64  
 6   PA              660 non-null    int64  
 7   AB              660 non-null    int64  
 8   R               660 non-null    int64  
 9   H               660 non-null    int64  
 10  2B              660 non-null    int64  
 11  3B              660 non-null    int64  
 12  HR              660 non-null    int64  
 13  RBI             660 non-null    int64  
 14  SB              660 non-null    int64  
 15  CS              660 non-null    int64  
 16  BB              660 non-null    int64  
 17  SO              660 non-null    int

In [60]:
df['predicted_runs'] = df.apply(predicted_run, axis=1)

In [67]:
df['predicted_runs_v2'] = df.apply(predicted_run_v2, axis=1)

In [68]:
df['absolute_error_v2'] = df.apply(lambda row: row['R']-row['predicted_runs_v2'], axis=1)

In [62]:
df['absolute_error'] = df.apply(lambda row: row['R']-row['predicted_runs'], axis=1)

In [63]:
df.head()

,YEAR,TM,#BAT,BATAGE,RPG,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,BA,OBP,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,LOB,BB+HBP,1B,predicted_runs,absolute_error
0,2000,Anaheim Angels,45,27.6,5.33,162,6373,5628,864,1574,309,34,236,837,93,52,608,1024,0.280,0.352,0.472,0.825,105,2659,126,47,47,43,43,1173,655,995,862.628,1.372
1,2000,Arizona Diamondbacks,41,30.8,4.89,162,6241,5527,792,1466,282,44,179,756,97,44,535,975,0.265,0.333,0.429,0.763,88,2373,114,59,61,58,37,1128,594,961,764.078,27.922
2,2000,Atlanta Braves,47,30.8,5.00,162,6275,5489,810,1490,274,26,179,758,148,56,595,1010,0.271,0.346,0.429,0.775,95,2353,127,59,87,45,38,1192,654,1011,773.051,36.949
3,2000,Baltimore Orioles,50,32.1,4.90,162,6238,5549,794,1508,310,22,184,750,126,65,558,900,0.272,0.341,0.435,0.776,100,2414,148,49,27,54,34,1129,607,992,772.801,21.199
4,2000,Boston Red Sox,52,29.3,4.89,162,6371,5630,792,1503,316,32,167,755,43,30,611,1019,0.267,0.341,0.423,0.764,90,2384,115,42,40,48,40,1226,653,988,777.605,14.395


In [64]:
np.mean(abs(df['absolute_error']))

25.89906515151516

In [70]:
np.mean(abs(df['absolute_error_v2']))

30.56683787878787